In [1]:
# Import Needed Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [2]:
for dir_name, _, filenames in os.walk("/Users/rish/Desktop/Mini-Projects/ML Projects/Data/archive"):
    for filename in filenames:
        print(os.path.join(dir_name, filename))

In [3]:
import matplotlib.pyplot as plt
import pathlib, random, math
import splitfolders
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *

In [4]:
%mkdir fish_data_dir
%cd fish_data_dir
%mkdir Catla Silver Gulfaam Grass

%cp /Users/rish/Desktop/Mini-Projects/MLProjects/Data/archive/train/Catla*.JPG Catla/
%cp /Users/rish/Desktop/Mini-Projects/MLProjects/Data/archive/train/Silver*.JPG Silver/
%cp /Users/rish/Desktop/Mini-Projects/MLProjects/Data/archive/train/Gulfaam*.JPG Gulfaam/
%cp /Users/rish/Desktop/Mini-Projects/MLProjects/Data/archive/train/Grass*.JPG Grass/

%ls

/Users/rish/Desktop/Mini-Projects/MLProjects/fish_data_dir
Catla/   Grass/   Gulfaam/ Silver/


In [4]:
epochs = 30
batch_size = 16
img_hieght, img_width = 300, 300
input_shape = (img_hieght, img_width, 3)

In [5]:
def create_data_sparse(data_bs):
    data_bs = pathlib.Path(data_bs)
    splitfolders.ratio(data_bs, output="Img/", seed = 42, ratio = (0.7, 0.15, 0.15), group_prefix=None)
    data_gen = ImageDataGenerator(rescale=1.0/255)
    train_ds = data_gen.flow_from_directory("Img/train/", target_size = (img_hieght, img_width), class_mode = 'sparse', 
                                            batch_size = batch_size, subset = "training")
    val_ds = data_gen.flow_from_directory("Img/val/", target_size = (img_hieght, img_width), class_mode = 'sparse', 
                                            batch_size = batch_size, shuffle = False)
    
    return train_ds, val_ds


In [6]:
train_ds, val_ds = create_data_sparse("/Users/rish/Desktop/Mini-Projects/MLProjects/fish_data_dir")

Copying files: 119 files [00:01, 110.14 files/s]

Found 81 images belonging to 5 classes.
Found 16 images belonging to 5 classes.


In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu", input_shape = input_shape),
    tf.keras.layers.Conv2D(16, (2,2), activation = "relu"),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(100, activation = "relu"),
    tf.keras.layers.Dense(30, activation = "relu"),

    tf.keras.layers.Dense(1, activation = "softmax")
])

In [15]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer = "adam", 
metrics = ["accuracy"])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 298, 298, 32)      896       
                                                                 
 conv2d_4 (Conv2D)           (None, 297, 297, 16)      2064      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 148, 148, 16)     0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 350464)            0         
                                                                 
 dense (Dense)               (None, 100)               35046500  
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                        

In [17]:
model.fit(train_ds, epochs = epochs, batch_size = batch_size)

Epoch 1/30


2022-07-31 15:34:29.679801: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 4 which is outside the valid range of [0, 1).  Label values: 2 4 4 4 2 4 4 0 0 4 2 4 2 4 2 4


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/Users/rish/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/rish/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/rish/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/Users/rish/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/rish/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/Users/rish/opt/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
     